# 第6章：预认知（逐步思考）

- [课程](#lesson)
- [练习](#exercises)
- [示例演练场](#example-playground)

## 设置

运行以下设置单元格以加载您的API密钥并建立`get_completion`辅助函数。

In [ ]:
%pip install anthropic

# 导入 Python 内置的正则表达式库
import re
import anthropic

# 从 IPython 存储中检索 API_KEY 和 MODEL_NAME 变量
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## 课程

如果有人把你叫醒，然后立刻开始问你几个复杂的问题，并且你必须马上回答，你会表现如何？可能不如你先有时间**仔细思考你的答案**那样好。

你猜怎么着？Claude 也是如此。

**给 Claude 时间逐步思考有时会使其更准确**，特别是对于复杂的任务。然而，**思考只有在“大声说出来”时才算数**。你不能要求 Claude 思考但只输出答案——在这种情况下，实际上并没有发生思考。

### 示例

在下面的提示中，人类读者很清楚第二句话与第一句话是矛盾的。但 **Claude 对“不相关”这个词理解得过于字面化了**。

In [ ]:
# 提示
PROMPT = """这部电影评论的情绪是积极的还是消极的？

这部电影以其新颖和原创性让我大开眼界。在完全不相关的消息中，我自1900年以来一直生活在石头下。"""

# 打印 Claude 的回应
print(get_completion(PROMPT))

为了改进 Claude 的回应，让我们**允许 Claude 在回答之前先进行思考**。我们通过字面化地列出 Claude 在处理和思考任务时应该采取的步骤来实现这一点。再加上一点角色提示，这能让 Claude 更深入地理解评论。

In [ ]:
# 系统提示
SYSTEM_PROMPT = "你是一个精明的电影评论读者。"

# 提示
PROMPT = """这部评论的情绪是积极的还是消极的？首先，用 <positive-argument> 和 <negative-argument> XML 标签写出每一方的最佳论据，然后回答。

这部电影以其新颖和原创性让我大开眼界。在完全不相关的消息中，我自1900年以来一直生活在石头下。"""

# 打印 Claude 的回应
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude 有时对顺序很敏感**。这个例子处于 Claude 理解细微文本能力的边界，当我们交换上一个例子中论据的顺序，将消极论据放在第一位，积极论据放在第二位时，这会将 Claude 的整体评估改变为积极。

在大多数情况下（但并非所有情况，这令人困惑），**Claude 更倾向于选择两个选项中的第二个**，这可能是因为它在网络训练数据中，第二个选项更有可能是正确的。

In [ ]:
# 提示
PROMPT = """这部评论的情绪是消极的还是积极的？首先，用 <negative-argument> 和 <positive-argument> XML 标签写出每一方的最佳论据，然后回答。

这部电影以其新颖和原创性让我大开眼界。不相关地，我自1900年以来一直生活在石头下。"""

# 打印 Claude 的回应
print(get_completion(PROMPT))

**让 Claude 思考可以将 Claude 的答案从不正确变为正确**。在许多 Claude 犯错的情况下，就是这么简单！

让我们来看一个 Claude 答案不正确的例子，看看如何通过要求 Claude 思考来纠正它。

In [ ]:
# 提示
PROMPT = "说出一部由1956年出生的演员主演的著名电影。"

# 打印 Claude 的回应
print(get_completion(PROMPT))

让我们通过要求 Claude 逐步思考来解决这个问题，这次是在 `<brainstorm>` 标签中。

In [ ]:
# 提示
PROMPT = "说出一部由1956年出生的演员主演的著名电影。首先在 <brainstorm> 标签中头脑风暴一些演员及其出生年份，然后给出你的答案。"

# 打印 Claude 的回应
print(get_completion(PROMPT))

如果您想在不更改上方任何内容的情况下，尝试本课程的提示，请一直滚动到课程笔记本的底部，访问[**示例演练场**](#example-playground)。

---

## 练习
- [练习 6.1 - 邮件分类](#exercise-61---classifying-emails)
- [练习 6.2 - 邮件分类格式化](#exercise-62---email-classification-formatting)

### 练习 6.1 - 邮件分类
在这个练习中，我们将指导 Claude 将邮件分类到以下类别：
- (A) 售前问题
- (B) 损坏或有缺陷的商品
- (C) 账单问题
- (D) 其他（请解释）

对于练习的第一部分，请修改 `PROMPT`，**让 Claude 输出正确的分类，并且只输出分类**。您的答案需要**包含正确选项的字母（A - D），带括号，以及类别的名称**。

请参考 `EMAILS` 列表中每封邮件旁边的注释，以了解该邮件应归入哪个类别。

In [ ]:
# 提示模板，带有变量内容的占位符
PROMPT = """请将此邮件分类为绿色或蓝色：{email}"""

# Claude 回应的预填充内容（如果有的话）
PREFILL = ""

# 变量内容以列表形式存储
EMAILS = [
    "嗨——我的 Mixmaster4000 在运行时发出奇怪的噪音。它还有点烟熏味和塑料味，像电子设备烧焦了。我需要一个替换品。", # (B) 损坏或有缺陷的商品
    "我可以用我的 Mixmaster 4000 搅拌油漆吗，还是它只用于搅拌食物？", # (A) 售前问题 或 (D) 其他（请解释）
    "我取消后已经等了4个月，我的月费还没停！！到底怎么回事？？？", # (C) 账单问题
    "我怎么到这儿来的，我不擅长电脑。救命。" # (D) 其他（请解释）
]

# 正确分类以列表的列表形式存储，以适应每封邮件可能存在多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 用于正则表达式评分的每个类别的字符串值字典
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# 遍历邮件列表
for i,email in enumerate(EMAILS):
    
    # 将邮件文本替换到邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取 Claude 的回应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 评估 Claude 的回应
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # 打印 Claude 的回应
    print("--------------------------- 包含变量替换的完整提示 ---------------------------")
    print("用户回合")
    print(formatted_prompt)
    print("\n助手回合")
    print(PREFILL)
    print("\n------------------------------------- Claude 的回应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("本练习已正确解决：", grade, "\n\n\n\n\n\n")

❓ 如果您想要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

还是卡住了？运行下面的单元格查看示例解决方案。

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### 练习 6.2 - 邮件分类格式化
在这个练习中，我们将优化上述提示的输出，使其产生我们想要的精确格式的答案。

使用您最喜欢的输出格式化技术，让 Claude **只将正确分类的字母**用 `<answer></answer>` 标签包裹起来。例如，第一封邮件的答案应该包含精确的字符串 `<answer>B</answer>`。

如果您忘记了每封邮件对应的正确字母类别，请参考 `EMAILS` 列表中每封邮件旁边的注释。

In [ ]:
# 提示模板，带有变量内容的占位符
PROMPT = """请将此邮件分类为绿色或蓝色：{email}"""

# Claude 回应的预填充内容（如果有的话）
PREFILL = ""

# 变量内容以列表形式存储
EMAILS = [
    "嗨——我的 Mixmaster4000 在运行时发出奇怪的噪音。它还有点烟熏味和塑料味，像电子设备烧焦了。我需要一个替换品。", # (B) 损坏或有缺陷的商品
    "我可以用我的 Mixmaster 4000 搅拌油漆吗，还是它只用于搅拌食物？", # (A) 售前问题 或 (D) 其他（请解释）
    "我取消后已经等了4个月，我的月费还没停！！到底怎么回事？？？", # (C) 账单问题
    "我怎么到这儿来的，我不擅长电脑。救命。" # (D) 其他（请解释）
]

# 正确分类以列表的列表形式存储，以适应每封邮件可能存在多个正确分类的情况
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# 用于正则表达式评分的每个类别的字符串值字典
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# 遍历邮件列表
for i,email in enumerate(EMAILS):
    
    # 将邮件文本替换到邮件占位符变量中
    formatted_prompt = PROMPT.format(email=email)
   
    # 获取 Claude 的回应
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # 评估 Claude 的回应
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # 打印 Claude 的回应
    print("--------------------------- 包含变量替换的完整提示 ---------------------------")
    print("用户回合")
    print(formatted_prompt)
    print("\n助手回合")
    print(PREFILL)
    print("\n------------------------------------- Claude 的回应 -------------------------------------")
    print(response)
    print("\n------------------------------------------ 评分 ------------------------------------------")
    print("本练习已正确解决：", grade, "\n\n\n\n\n\n")

❓ 如果您想要提示，请运行下面的单元格！

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### 恭喜！

如果您已经解决了到目前为止的所有练习，您就可以进入下一章了。祝您提示愉快！

---

# Chapter 6: Precognition (Thinking Step by Step)

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
%pip install anthropic

# Import python's built-in regular expression library
import re
import anthropic

# Retrieve the API_KEY & MODEL_NAME variables from the IPython store
%store -r API_KEY
%store -r MODEL_NAME

client = anthropic.Anthropic(api_key=API_KEY)

def get_completion(prompt: str, system_prompt="", prefill=""):
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=2000,
        temperature=0.0,
        system=system_prompt,
        messages=[
          {"role": "user", "content": prompt},
          {"role": "assistant", "content": prefill}
        ]
    )
    return message.content[0].text

---

## Lesson

If someone woke you up and immediately started asking you several complicated questions that you had to respond to right away, how would you do? Probably not as good as if you were given time to **think through your answer first**. 

Guess what? Claude is the same way.

**Giving Claude time to think step by step sometimes makes Claude more accurate**, particularly for complex tasks. However, **thinking only counts when it's out loud**. You cannot ask Claude to think but output only the answer - in this case, no thinking has actually occurred.

### Examples

In the prompt below, it's clear to a human reader that the second sentence belies the first. But **Claude takes the word "unrelated" too literally**.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

To improve Claude's response, let's **allow Claude to think things out first before answering**. We do that by literally spelling out the steps that Claude should take in order to process and think through its task. Along with a dash of role prompting, this empowers Claude to understand the review more deeply.

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

**Claude is sometimes sensitive to ordering**. This example is on the frontier of Claude's ability to understand nuanced text, and when we swap the order of the arguments from the previous example so that negative is first and positive is second, this changes Claude's overall assessment to positive.

In most situations (but not all, confusingly enough), **Claude is more likely to choose the second of two options**, possibly because in its training data from the web, second options were more likely to be correct.

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

**Letting Claude think can shift Claude's answer from incorrect to correct**. It's that simple in many cases where Claude makes mistakes!

Let's go through an example where Claude's answer is incorrect to see how asking Claude to think can fix that.

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

Let's fix this by asking Claude to think step by step, this time in `<brainstorm>` tags.

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 6.1 - Classifying Emails](#exercise-61---classifying-emails)
- [Exercise 6.2 - Email Classification Formatting](#exercise-62---email-classification-formatting)

### Exercise 6.1 - Classifying Emails
In this exercise, we'll be instructing Claude to sort emails into the following categories:										
- (A) Pre-sale question
- (B) Broken or defective item
- (C) Billing question
- (D) Other (please explain)

For the first part of the exercise, change the `PROMPT` to **make Claude output the correct classification and ONLY the classification**. Your answer needs to **include the letter (A - D) of the correct choice, with the parentheses, as well as the name of the category**.

Refer to the comments beside each email in the `EMAILS` list to know which category that email should be classified under.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "A\) P",
    "B": "B\) B",
    "C": "C\) B",
    "D": "D\) O"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_6_1_hint; print(exercise_6_1_hint)

Still stuck? Run the cell below for an example solution.						

In [ ]:
from hints import exercise_6_1_solution; print(exercise_6_1_solution)

### Exercise 6.2 - Email Classification Formatting
In this exercise, we're going to refine the output of the above prompt to yield an answer formatted exactly how we want it. 

Use your favorite output formatting technique to make Claude wrap JUST the letter of the correct classification in `<answer></answer>` tags. For instance, the answer to the first email should contain the exact string `<answer>B</answer>`.

Refer to the comments beside each email in the `EMAILS` list if you forget which letter category is correct for each email.

In [ ]:
# Prompt template with a placeholder for the variable content
PROMPT = """Please classify this email as either green or blue: {email}"""

# Prefill for Claude's response, if any
PREFILL = ""

# Variable content stored as a list
EMAILS = [
    "Hi -- My Mixmaster4000 is producing a strange noise when I operate it. It also smells a bit smoky and plasticky, like burning electronics.  I need a replacement.", # (B) Broken or defective item
    "Can I use my Mixmaster 4000 to mix paint, or is it only meant for mixing food?", # (A) Pre-sale question OR (D) Other (please explain)
    "I HAVE BEEN WAITING 4 MONTHS FOR MY MONTHLY CHARGES TO END AFTER CANCELLING!!  WTF IS GOING ON???", # (C) Billing question
    "How did I get here I am not good with computer.  Halp." # (D) Other (please explain)
]

# Correct categorizations stored as a list of lists to accommodate the possibility of multiple correct categorizations per email
ANSWERS = [
    ["B"],
    ["A","D"],
    ["C"],
    ["D"]
]

# Dictionary of string values for each category to be used for regex grading
REGEX_CATEGORIES = {
    "A": "<answer>A</answer>",
    "B": "<answer>B</answer>",
    "C": "<answer>C</answer>",
    "D": "<answer>D</answer>"
}

# Iterate through list of emails
for i,email in enumerate(EMAILS):
    
    # Substitute the email text into the email placeholder variable
    formatted_prompt = PROMPT.format(email=email)
   
    # Get Claude's response
    response = get_completion(formatted_prompt, prefill=PREFILL)

    # Grade Claude's response
    grade = any([bool(re.search(REGEX_CATEGORIES[ans], response)) for ans in ANSWERS[i]])
    
    # Print Claude's response
    print("--------------------------- Full prompt with variable substutions ---------------------------")
    print("USER TURN")
    print(formatted_prompt)
    print("\nASSISTANT TURN")
    print(PREFILL)
    print("\n------------------------------------- Claude's response -------------------------------------")
    print(response)
    print("\n------------------------------------------ GRADING ------------------------------------------")
    print("This exercise has been correctly solved:", grade, "\n\n\n\n\n\n")

❓ If you want a hint, run the cell below!

In [ ]:
from hints import exercise_6_2_hint; print(exercise_6_2_hint)

### Congrats!

If you've solved all exercises up until this point, you're ready to move to the next chapter. Happy prompting!

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect Claude's responses.

In [ ]:
# Prompt
PROMPT = """Is this movie review sentiment positive or negative?

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since the year 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# System prompt
SYSTEM_PROMPT = "You are a savvy reader of movie reviews."

# Prompt
PROMPT = """Is this review sentiment positive or negative? First, write the best arguments for each side in <positive-argument> and <negative-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. In totally unrelated news, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

In [ ]:
# Prompt
PROMPT = """Is this review sentiment negative or positive? First write the best arguments for each side in <negative-argument> and <positive-argument> XML tags, then answer.

This movie blew my mind with its freshness and originality. Unrelatedly, I have been living under a rock since 1900."""

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956."

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Name a famous movie starring an actor who was born in the year 1956. First brainstorm about some actors and their birth years in <brainstorm> tags, then give your answer."

# Print Claude's response
print(get_completion(PROMPT))